In [1]:
from utils.dataloader import dtLoader

In [5]:
source = "/Users/pradithaalwis/Projects/University of Peradeniya/Fetal Mov Data/Ward/"
drop_zero_path = "/Users/pradithaalwis/Projects/University of Peradeniya/Fetal Mov Data//Randomly_Dropped_Windows/"
sensors = ['ax1', 'ay1', 'az1', 'ax2', 'ay2', 'az2', 'ax3', 'ay3', 'az3', 'ax4', 'ay4', 'az4']
sample_freq = 32

# Stride in seconds
stride = 1

# Window length in seconds
window_length = 8

# Safety margin in seconds
front_margin = 2
rear_margin = 2

In [6]:
"""
Splits the signals into windows
Args:
    SOURCE          : path to the sensor data
    SAMPLE_FREQ     : sample frequency of sensors
    STRIDE          : stride used for windowing (in seconds)
    WINDOW_LENGTH   : length of the required window in seconds
    FRONT_MARGIN    : Required offset before a fetal kick (in seconds)
    REAR_MARGIN     : Required offset after a fetal kick (in seconds)
"""
dataloader = dtLoader(SOURCE=source,
                        SAMPLE_FREQ=sample_freq,
                        STRIDE=stride,
                        WINDOW_LENGTH=window_length,
                        FRONT_MARGIN=front_margin,
                        REAR_MARGIN=rear_margin,
                        SENSORS=sensors)

In [7]:
"""
Args:
Returns:
    self.window_list            : Set of generated windows
    self.kick_count_list        : Number of kicks in each window
    counts                      : Dictionary containing number of windows                             with the given number of kicks
"""
counts = dataloader.split_windows()

--------------------------------------------------------------------------------
These are the list of files in the source directory.
['W17_R1.csv', 'W09_R1.csv', 'W05_R1.csv', 'W21_R1.csv', 'W05_R2.csv', 'W21_R2.csv', 'W15_R2.csv', 'W19_R1.csv', 'W07_R1.csv', 'W15_R1.csv', 'W23_R1.csv', 'W11_R3.csv', 'W27_R2.csv', 'W11_R2.csv', 'W27_R1.csv', 'W11_R1.csv', 'W03_R1.csv', 'W25_R1.csv', 'W11_R5.csv', 'W29_R1.csv', 'W01_R1.csv', 'W11_R4.csv', 'W13_R1.csv', 'W25_R2.csv', 'W29_R2.csv', 'W29_R3.csv', 'W25_R3.csv', 'W04_R2.csv', 'W20_R1.csv', 'W08_R1.csv', 'W16_R1.csv', 'W04_R1.csv', 'W22_R1.csv', 'W30_R1.csv', 'W06_R1.csv', 'W18_R1.csv', 'W14_R1.csv', 'W22_R2.csv', 'W10_R1.csv', 'W02_R1.csv', 'W26_R1.csv', 'W24_R3.csv', 'W24_R2.csv', 'W12_R1.csv', 'W24_R1.csv', 'W28_R1.csv']
--------------------------------------------------------------------------------
Length of dataframe :  1286  seconds
W17_R1.csv has been processed
Length of dataframe :  1263  seconds
W09_R1.csv has been processed
Length

In [8]:
window_list, num_kick_list = dataloader.drop_zero_kicks(counts[0] - counts[1], drop_zero_path)

In [9]:
print(window_list.shape)

(3640, 12, 256)
